<a href="https://colab.research.google.com/github/GreatLearningAIML1/chennai-nov18-batch-sharan19/blob/master/R6_Internal_Lab_UpdatedTF2_Prices_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [0]:
#from google.colab import drive
#drive.mount('/gdrive')
from sklearn.preprocessing import Normalizer

In [0]:
import pandas as pd

In [5]:
data = pd.read_csv('prices.csv')
data.shape

(851264, 7)

### Check all columns in the dataset

In [6]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis=1,inplace=True)


In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [9]:
data.volume=data.volume/1000
data_1000=data.iloc[:1000,:]
data_1000.shape

(1000, 5)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X=data_1000.drop('volume',axis=1)
Y=data_1000.pop('volume')


In [0]:
transformer = Normalizer()

In [13]:
X = transformer.fit_transform(X)
X.shape

(1000, 4)

In [0]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.3,random_state=1)

In [0]:
Xtrain = Xtrain.astype('float32')
ytrain = ytrain.astype('float32')
Xtest  = Xtest.astype('float32')
ytest  = ytest.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [16]:
import numpy as np
ytrain=np.array([ytrain]).transpose()
#ytest=np.array(ytest).transpose
ytrain.shape

(700, 1)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [21]:
for i in range(100):
    
    w, b = train(Xtrain, ytrain, w, b)
    print('Current Loss on iteration', i, loss(ytrain, prediction(Xtrain, w, b)).numpy())
    
print('Loss at every 5th iteration is:')
for j in range(100):
    w, b = train(Xtrain, ytrain, w, b)
    if j%5==0:
      print('Current Loss on iteration', j, loss(ytrain, prediction(Xtrain, w, b)).numpy())

('Current Loss on iteration', 0, 223490370.0)
('Current Loss on iteration', 1, 221288480.0)
('Current Loss on iteration', 2, 219259230.0)
('Current Loss on iteration', 3, 217389040.0)
('Current Loss on iteration', 4, 215665470.0)
('Current Loss on iteration', 5, 214077070.0)
('Current Loss on iteration', 6, 212613100.0)
('Current Loss on iteration', 7, 211263980.0)
('Current Loss on iteration', 8, 210020590.0)
('Current Loss on iteration', 9, 208874700.0)
('Current Loss on iteration', 10, 207818660.0)
('Current Loss on iteration', 11, 206845360.0)
('Current Loss on iteration', 12, 205948400.0)
('Current Loss on iteration', 13, 205121760.0)
('Current Loss on iteration', 14, 204359900.0)
('Current Loss on iteration', 15, 203657760.0)
('Current Loss on iteration', 16, 203010670.0)
('Current Loss on iteration', 17, 202414350.0)
('Current Loss on iteration', 18, 201864740.0)
('Current Loss on iteration', 19, 201358240.0)
('Current Loss on iteration', 20, 200891440.0)
('Current Loss on itera

### Get the shapes and values of W and b

In [22]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

('Weights:\n', array([[1325.5935],
       [1423.9921],
       [1328.6841],
       [1438.602 ]], dtype=float32))
('Bias:\n', array([2759.8713], dtype=float32))


### Model Prediction on 1st Examples in Test Dataset

In [23]:
prediction([Xtest[0]], w, b)

<tf.Tensor: id=8091, shape=(1, 1), dtype=float32, numpy=array([[5518.8154]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [24]:
df = pd.read_csv('Iris.csv')
df.shape

(150, 6)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
df = pd.get_dummies(df, columns=['Species'])

In [26]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
x=df.iloc[:,1:5]
y=df.iloc[:,5:8]

In [28]:

x.shape

(150, 4)

In [29]:
y.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
#x.drop(['Id'],axis=1,inplace=True)

In [0]:
x = transformer.fit_transform(x)
#x.head()

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
xtrain1,xtest1,ytrain1,ytest1=train_test_split(x,y,test_size=0.3,random_state=1)

### Model Training 

In [34]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10,input_dim=4),
    tf.keras.layers.Dense(3,activation='softmax')
])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                50        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 33        
Total params: 83
Trainable params: 83
Non-trainable params: 0
_________________________________________________________________


### Model Prediction

In [0]:
model.add(tf.keras.layers.BatchNormalization())

In [36]:
#model.add(tf.keras.layers.Dense(100))
xtrain1.shape

(105, 4)

In [37]:
ytrain1.shape
#model.add(tf.keras.layers.Dense(100, activation='softmax'))
#model.add(tf.keras.layers.Dense(3,activation='softmax'))

(105, 3)

In [0]:
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

In [0]:
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
#model.summary()

In [44]:
model.fit(xtrain1,ytrain1,          
          validation_data=(xtest1,ytest1),
          epochs=50,
          batch_size=xtrain1.shape[0])

Train on 105 samples, validate on 45 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
105/105 [==============================] - 0s 2ms/sample - loss: 5.6854 - acc: 0.1429 - val_loss: 1.3537 - val_acc: 0.2889
Epoch 2/50
105/105 [==============================] - 0s 46us/sample - loss: 5.6803 - acc: 0.1524 - val_loss: 1.3548 - val_acc: 0.2889
Epoch 3/50
105/105 [==============================] - 0s 37us/sample - loss: 5.6797 - acc: 0.1619 - val_loss: 1.3555 - val_acc: 0.2889
Epoch 4/50
105/105 [==============================] - 0s 40us/sample - loss: 5.6797 - acc: 0.1619 - val_loss: 1.3562 - val_acc: 0.2889
Epoch 5/50
105/105 [==============================] - 0s 37us/sample - loss: 5.6797 - acc: 0.1619 - val_loss: 1.3570 - val_acc: 0.2889
Epoch 6/50
105/105 [==============================] - 0s 37us/sample - loss: 5.6797 - acc: 0.1619 - val_loss: 1.3578 - val_acc: 0.2889
Epoch 7/50
105/105 [==============================] - 0s 38us/sample - loss: 5.6797 - acc: 0.1619 

### Save the Model

In [0]:
model.save_weights('iris_weights.h5')

In [46]:
model.save('iris.h5')

RuntimeError: ignored

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?